In [37]:
import torch
from torchvision import transforms, datasets, models
import torch.nn as nn

In [38]:
def double_conv2D(in_channel,out_channel):
    conv= nn.Sequential(
        nn.Conv2d(in_channel,out_channel,kernel_size=3,padding=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channel,out_channel,kernel_size=3,padding=1),
        nn.ReLU(inplace=True))
    return conv

In [39]:
def tripple_conv2D(in_channel,out_channel):
    conv= nn.Sequential(
        nn.Conv2d(in_channel,out_channel,kernel_size=3,padding=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channel,out_channel,kernel_size=3,padding=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channel,out_channel,kernel_size=3,padding=1),
        nn.ReLU(inplace=True))
    return conv

In [40]:
class FC(nn.Module):
    def __init__(self):
        super(FC,self).__init__()
        self.conv1= double_conv2D(1,64)
        self.maxpool2D_1 = nn.MaxPool2d(kernel_size=2,stride=2,padding=0)
        self.conv2= double_conv2D(64,128)
        self.maxpool2D_2 = nn.MaxPool2d(kernel_size=2,stride=2,padding=0)
        self.conv3= tripple_conv2D(128,256)
        self.maxpool2D_3 = nn.MaxPool2d(kernel_size=2,stride=2,padding=0)
        self.conv4= tripple_conv2D(256,512)
        self.maxpool2D_4 = nn.MaxPool2d(kernel_size=2,stride=2,padding=0)
        self.conv5= tripple_conv2D(512,512)
        self.maxpool2D_5 = nn.MaxPool2d(kernel_size=2,stride=2,padding=0)
        self.conv6= nn.Sequential(nn.Conv2d(512,4096,kernel_size=3,padding=1),nn.ReLU(inplace=True))
        self.conv7= nn.Sequential(nn.Conv2d(4096,4096,kernel_size=1,padding=0),nn.ReLU(inplace=True))
        
        self.score5=nn.Sequential(nn.Conv2d(4096,2,kernel_size=1,padding=0),nn.ReLU(inplace=True))
        self.score4=nn.Sequential(nn.Conv2d(512,2,kernel_size=1,padding=0),nn.ReLU(inplace=True))
        self.score3=nn.Sequential(nn.Conv2d(256,2,kernel_size=1,padding=0),nn.ReLU(inplace=True))        
        
        self.upscore1 = nn.ConvTranspose2d(2,2,kernel_size=4,stride=2,padding=1)
        self.upscore2 = nn.ConvTranspose2d(2,2,kernel_size=4,stride=2,padding=1)
        self.upscore3 = nn.ConvTranspose2d(2,2,kernel_size=16,stride=8,padding=4)
        
    def forward(self,img):
        #encorder
        x1 = self.conv1(img)

        x2 = self.maxpool2D_1(x1)

        x3 = self.conv2(x2)
        x4 = self.maxpool2D_2(x3)
        x5 = self.conv3(x4)
        x6 = self.maxpool2D_3(x5)
        x7 = self.conv4(x6)
        x8 = self.maxpool2D_4(x7)
        x9 = self.conv5(x8)
        x10 = self.maxpool2D_5(x9)
        x11 = self.conv6(x10)
        print(x11.size())
        x12 = self.conv7(x11)
        print(x12.size())
        
        s5 = self.score5(x12)
        print(s5.size())
        s4 = self.score4(x8)
        s3 = self.score3(x6)
        
        us1= self.upscore1(s5)
        summ1 = us1+s4
        us2 = self.upscore2(summ1)
        summ2= us2+s3
        us3 = self.upscore3(summ2)
        return us3

In [41]:
#for testing
img = torch.rand((4,1,224,224))
model = FC()
model(img)

torch.Size([4, 4096, 7, 7])
torch.Size([4, 4096, 7, 7])
torch.Size([4, 2, 7, 7])


tensor([[[[-0.0346, -0.0332, -0.0391,  ..., -0.0387, -0.0330, -0.0296],
          [-0.0341, -0.0315, -0.0368,  ..., -0.0283, -0.0372, -0.0393],
          [-0.0315, -0.0306, -0.0311,  ..., -0.0294, -0.0302, -0.0319],
          ...,
          [-0.0322, -0.0296, -0.0342,  ..., -0.0379, -0.0331, -0.0323],
          [-0.0338, -0.0320, -0.0343,  ..., -0.0369, -0.0333, -0.0342],
          [-0.0321, -0.0359, -0.0349,  ..., -0.0310, -0.0333, -0.0331]],

         [[-0.0268, -0.0342, -0.0364,  ..., -0.0366, -0.0333, -0.0259],
          [-0.0349, -0.0316, -0.0372,  ..., -0.0271, -0.0320, -0.0259],
          [-0.0256, -0.0261, -0.0304,  ..., -0.0343, -0.0269, -0.0323],
          ...,
          [-0.0346, -0.0322, -0.0310,  ..., -0.0375, -0.0283, -0.0295],
          [-0.0304, -0.0325, -0.0313,  ..., -0.0335, -0.0353, -0.0346],
          [-0.0336, -0.0288, -0.0307,  ..., -0.0319, -0.0258, -0.0262]]],


        [[[-0.0346, -0.0332, -0.0391,  ..., -0.0387, -0.0330, -0.0296],
          [-0.0341, -0.0315,

In [35]:
print(model)

FC(
  (conv1): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
  )
  (maxpool2D_1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
  )
  (maxpool2D_2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU(inplace=True)
  )
  

In [36]:
model(img).shape

torch.Size([4, 4096, 7, 7])
torch.Size([4, 4096, 7, 7])
torch.Size([4, 2, 7, 7])


torch.Size([4, 2, 224, 224])

In [9]:
nn.ZeroPad2d(92)

ZeroPad2d(padding=(92, 92, 92, 92), value=0.0)